In [ ]:
import networkx as nx
import nltk
import json
import math
from tqdm import tqdm
import numpy as np
import sys

In [ ]:
def construct_graph(cpnet_csv_path, cpnet_vocab_path, output_path, prune=True):
    print('generating ConceptNet graph file...')
    concept2id = {}
    id2concept = {}
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        id2concept = [w.strip() for w in fin]
    concept2id = {w: i for i, w in enumerate(id2concept)}
    print(concept2id)

    id2relation = merged_relations
    relation2id = {r: i for i, r in enumerate(id2relation)}

    graph = nx.MultiDiGraph()
    nrow = sum(1 for _ in open(cpnet_csv_path, 'r', encoding='utf-8'))
    with open(cpnet_csv_path, "r", encoding="utf8") as fin:

        def not_save(cpt):
            return False

        attrs = set()

        for line in tqdm(fin, total=nrow):
            ls = line.strip().split('\t')
            rel = relation2id[ls[0]]
            subj = concept2id[ls[1]]
            obj = concept2id[ls[2]]
            weight = float(ls[3])
            if prune and (not_save(ls[1]) or not_save(ls[2]) or id2relation[rel] == "hascontext"):
                continue
            # if id2relation[rel] == "relatedto" or id2relation[rel] == "antonym":
            # weight -= 0.3
            # continue
            if subj == obj:  # delete loops
                continue
            # weight = 1 + float(math.exp(1 - weight))  # issue: ???

            if (subj, obj, rel) not in attrs:
                graph.add_edge(subj, obj, rel=rel, weight=weight)
                attrs.add((subj, obj, rel))
                graph.add_edge(obj, subj, rel=rel + len(relation2id), weight=weight)
                attrs.add((obj, subj, rel + len(relation2id)))

    nx.write_gpickle(graph, output_path)
    print(f"graph file saved to {output_path}")
    print()

In [ ]:
merged_relations = [
    'antonym',
    'atlocation',
    'capableof',
    'causes',
    'createdby',
    'isa',
    'desires',
    'hassubevent',
    'partof',
    'hascontext',
    'hasproperty',
    'madeof',
    'notcapableof',
    'notdesires',
    'receivesaction',
    'relatedto',
    'usedfor',
] 

In [ ]:
cpnet_csv_path = '../input/raw-chineseconceptnet/conceptnet.zh.csv'
cpnet_vocab_path = '../input/raw-chineseconceptnet/concept.txt'
output_path = 'conceptnet.zh.unpruned.graph'
prune=False
construct_graph(cpnet_csv_path, cpnet_vocab_path, output_path, prune)

In [ ]:
concept2id = {}
id2concept = {}
with open('../input/raw-chineseconceptnet/concept.txt', "r", encoding="utf8") as fin:
    id2concept = [w.strip() for w in fin]

print(len(id2concept))
    
with open('concept2id.txt', 'w+') as fout:
    for i, w in enumerate(id2concept):
        fout.write(str(w) + '\t' + str(i) + '\n')
